#1.-QUANTUM ANNEALING

###Setup

In [1]:
pip install dwave-ocean-sdk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 110 kB 36.4 MB/s 
     |████████████████████████████████| 74 kB 2.5 MB/s 
     |████████████████████████████████| 700 kB 36.0 MB/s 
     |████████████████████████████████| 497 kB 51.5 MB/s 
     |████████████████████████████████| 5.3 MB 44.7 MB/s 
     |████████████████████████████████| 97 kB 7.2 MB/s 
     |████████████████████████████████| 90 kB 9.4 MB/s 
     |████████████████████████████████| 966 kB 56.6 MB/s 
     |████████████████████████████████| 11.8 MB 38.1 MB/s 
     |████████████████████████████████| 658 kB 49.7 MB/s 
     |████████████████████████████████| 44 kB 2.9 MB/s 
     |████████████████████████████████| 246 kB 56.9 MB/s 


In [2]:
!dwave config create

Using the simplified configuration flow.
Try 'dwave config create --full' for more options.

Creating new configuration file: /root/.config/dwave/dwave.conf
Profile [defaults]: 
Updating existing profile: defaults
Authentication token [skip]: DEV-9f527baff6a3722559445b822dcceee8e7aaf05f
Configuration saved.


##1.1- A simple example

In [3]:
import dimod
J = {(0,1):1, (0,2):1}
h = {}
problem = dimod.BinaryQuadraticModel(h, J, 0.0, dimod.SPIN)
print("The problem we are going to solve is:")
print(problem)

The problem we are going to solve is:
BinaryQuadraticModel({0: 0.0, 1: 0.0, 2: 0.0}, {(1, 0): 1.0, (2, 0): 1.0}, 0.0, 'SPIN')


### Running on the annealer

In [4]:
from dwave.system import DWaveSampler
from dwave.system import EmbeddingComposite
sampler = EmbeddingComposite(DWaveSampler())
result = sampler.sample(problem, num_reads=10)
print("The solutions that we have obtained are")
print(result)

The solutions that we have obtained are
   0  1  2 energy num_oc. chain_.
0 +1 -1 -1   -2.0       5     0.0
1 -1 +1 +1   -2.0       5     0.0
['SPIN', 2 rows, 10 samples, 3 variables]


In [5]:
print(result.info['timing']['qpu_access_time'], "microseconds")

15940.96 microseconds


##1.2- Constrained problems

In [6]:
x0 = dimod.Binary("x0")
x1 = dimod.Binary("x1")
x2 = dimod.Binary("x2")
blp = dimod.ConstrainedQuadraticModel()
blp.set_objective(-5*x0+3*x1-2*x2)
blp.add_constraint(x0 + x2 <= 1, "First constraint")
blp.add_constraint(3*x0 -x1 + 3*x2 <= 4, "Second constraint")
print("Our variables are:")
print(blp.variables)
print("Our objective is:")
print(blp.objective)
print("Our constraints are:") 
print(blp.constraints)

Our variables are:
Variables(['x0', 'x1', 'x2'])
Our objective is:
QuadraticModel({'x0': -5.0, 'x1': 3.0, 'x2': -2.0}, {}, 0.0, {'x0': 'BINARY', 'x1': 'BINARY', 'x2': 'BINARY'}, dtype='float64')
Our constraints are:
{'First constraint': Le(BinaryQuadraticModel({'x0': 1.0, 'x2': 1.0}, {}, 0.0, 'BINARY'), 1), 'Second constraint': Le(BinaryQuadraticModel({'x0': 3.0, 'x1': -1.0, 'x2': 3.0}, {}, 0.0, 'BINARY'), 4)}


In [7]:
sample1 = {"x0":1, "x1":1, "x2":1}
print("The assignment is", sample1)
print("Its cost is", blp.objective.energy(sample1))
print("Is it feasible?",blp.check_feasible(sample1))
print("The violations of the constraints are")
print(blp.violations(sample1))

The assignment is {'x0': 1, 'x1': 1, 'x2': 1}
Its cost is -4.0
Is it feasible? False
The violations of the constraints are
{'First constraint': 1.0, 'Second constraint': 1.0}


In [8]:
sample2 = {"x0":0, "x1":0, "x2":1}
print("The assignment is", sample2)
print("Its cost is", blp.objective.energy(sample2))
print("Is it feasible?",blp.check_feasible(sample2))
print("The violations of the constraints are")
print(blp.violations(sample2))

The assignment is {'x0': 0, 'x1': 0, 'x2': 1}
Its cost is -2.0
Is it feasible? True
The violations of the constraints are
{'First constraint': 0.0, 'Second constraint': -1.0}


### Exact solver

In [9]:
solver = dimod.ExactCQMSolver()
solution = solver.sample_cqm(blp)
print("The list of assignments is")
print(solution)

The list of assignments is
  x0 x1 x2 energy num_oc. is_sat. is_fea.
6  1  0  1   -7.0       1 arra...   False
2  1  0  0   -5.0       1 arra...    True
7  1  1  1   -4.0       1 arra...   False
3  1  1  0   -2.0       1 arra...    True
4  0  0  1   -2.0       1 arra...    True
0  0  0  0    0.0       1 arra...    True
5  0  1  1    1.0       1 arra...    True
1  0  1  0    3.0       1 arra...    True
['INTEGER', 8 rows, 8 samples, 3 variables]


In [10]:
print(solution.first)

Sample(sample={'x0': 1, 'x1': 0, 'x2': 1}, energy=-7.0, num_occurrences=1, is_satisfied=array([False, False]), is_feasible=False)


In [11]:
feasible_sols = solution.filter(lambda s: s.is_feasible)
print(feasible_sols)
print()
print(feasible_sols.first)

  x0 x1 x2 energy num_oc. is_sat. is_fea.
2  1  0  0   -5.0       1 arra...    True
3  1  1  0   -2.0       1 arra...    True
4  0  0  1   -2.0       1 arra...    True
0  0  0  0    0.0       1 arra...    True
5  0  1  1    1.0       1 arra...    True
1  0  1  0    3.0       1 arra...    True
['INTEGER', 6 rows, 6 samples, 3 variables]

Sample(sample={'x0': 1, 'x1': 0, 'x2': 0}, energy=-5.0, num_occurrences=1, is_satisfied=array([ True,  True]), is_feasible=True)


### Running on the annealer

In [39]:
y0, y1 = dimod.Binaries(["y0", "y1"])
cqm = dimod.ConstrainedQuadraticModel()
cqm.set_objective(-2*y0-3*y1)
cqm.add_constraint(y0 + 2*y1 <= 2)
print(cqm)

Constrained quadratic model: 2 variables, 1 constraints, 4 biases

Objective
  -2*Binary('y0') - 3*Binary('y1')

Constraints
  cf7bafc: Binary('y0') + 2*Binary('y1') <= 2

Bounds



In [40]:
qubo, invert = dimod.cqm_to_bqm(cqm, lagrange_multiplier = 5)
print(qubo)

BinaryQuadraticModel({'y0': -17.0, 'y1': -23.0, 'slack_v620a64a6f13e4a49b26ac70fd9b71063_0': -15.0, 'slack_v620a64a6f13e4a49b26ac70fd9b71063_1': -15.0}, {('y1', 'y0'): 20.0, ('slack_v620a64a6f13e4a49b26ac70fd9b71063_0', 'y0'): 10.0, ('slack_v620a64a6f13e4a49b26ac70fd9b71063_0', 'y1'): 20.0, ('slack_v620a64a6f13e4a49b26ac70fd9b71063_1', 'y0'): 10.0, ('slack_v620a64a6f13e4a49b26ac70fd9b71063_1', 'y1'): 20.0, ('slack_v620a64a6f13e4a49b26ac70fd9b71063_1', 'slack_v620a64a6f13e4a49b26ac70fd9b71063_0'): 10.0}, 20.0, 'BINARY')


In [41]:
sampler = EmbeddingComposite(DWaveSampler())
result = sampler.sample(qubo, num_reads=10)
print("The solutions that we have obtained are")
print(result)

The solutions that we have obtained are
  slack_v620a64a6f13e4a49b26ac70fd9b71063_0 ... y1 energy num_oc. chain_.
0                                         0 ...  1   -3.0       1     0.0
1                                         0 ...  0   -2.0       4     0.0
2                                         1 ...  0   -2.0       4     0.0
3                                         0 ...  1    0.0       1     0.0
['BINARY', 4 rows, 10 samples, 4 variables]


In [42]:
samples = []
occurrences = []
for s in result.data():
    samples.append(invert(s.sample))
    occurrences.append(s.num_occurrences)
sampleset = dimod.SampleSet.from_samples_cqm(samples,cqm,
    num_occurrences=occurrences)
print("The solutions to the original problem are")
print(sampleset)

The solutions to the original problem are
  y0 y1 energy num_oc. is_sat. is_fea.
3  1  1   -5.0       1 arra...   False
0  0  1   -3.0       1 arra...    True
1  1  0   -2.0       4 arra...    True
2  1  0   -2.0       4 arra...    True
['INTEGER', 4 rows, 10 samples, 2 variables]


In [43]:
final_sols = sampleset.filter(lambda s: s.is_feasible)
final_sols = final_sols.aggregate()
print("The final solutions are")
print(final_sols)

The final solutions are
  y0 y1 energy num_oc. is_sat. is_fea.
0  0  1   -3.0       1 arra...    True
1  1  0   -2.0       8 arra...    True
['INTEGER', 2 rows, 9 samples, 2 variables]


##1.3- The annealers

In [17]:
from dwave.cloud import Client
for solver in Client.from_config().get_solvers():
    print(solver)

StructuredSolver(id='DW_2000Q_6')
BQMSolver(id='hybrid_binary_quadratic_model_version2')
DQMSolver(id='hybrid_discrete_quadratic_model_version1')
CQMSolver(id='hybrid_constrained_quadratic_model_version1')
StructuredSolver(id='Advantage_system6.1')
StructuredSolver(id='Advantage2_prototype1.1')
StructuredSolver(id='Advantage_system4.1')


In [18]:
from dwave.system import DWaveSampler
sampler=DWaveSampler(solver='DW_2000Q_6')
print("Name:",sampler.properties["chip_id"])
print("Number of qubits:",sampler.properties["num_qubits"])
print("Category:",sampler.properties["category"])
print("Supported problems:",sampler.properties["supported_problem_types"])
print("Topology:",sampler.properties["topology"])
print("Range of reads:",sampler.properties["num_reads_range"])

Name: DW_2000Q_6
Number of qubits: 2048
Category: qpu
Supported problems: ['ising', 'qubo']
Topology: {'type': 'chimera', 'shape': [16, 16, 4]}
Range of reads: [1, 10000]


In [19]:
from dwave.system import DWaveSampler
sampler=DWaveSampler(solver='Advantage_system4.1')
print("Name:",sampler.properties["chip_id"])
print("Number of qubits:",sampler.properties["num_qubits"])
print("Category:",sampler.properties["category"])
print("Supported problems:",sampler.properties["supported_problem_types"])
print("Topology:",sampler.properties["topology"])
print("Range of reads:",sampler.properties["num_reads_range"])

Name: Advantage_system4.1
Number of qubits: 5760
Category: qpu
Supported problems: ['ising', 'qubo']
Topology: {'type': 'pegasus', 'shape': [16]}
Range of reads: [1, 10000]


In [20]:
from dwave.system import DWaveSampler
sampler=DWaveSampler(solver='Advantage2_prototype1.1')
print("Name:",sampler.properties["chip_id"])
print("Number of qubits:",sampler.properties["num_qubits"])
print("Category:",sampler.properties["category"])
print("Supported problems:",sampler.properties["supported_problem_types"])
print("Topology:",sampler.properties["topology"])
print("Range of reads:",sampler.properties["num_reads_range"])

Name: Advantage2_prototype1.1
Number of qubits: 576
Category: qpu
Supported problems: ['ising', 'qubo']
Topology: {'type': 'zephyr', 'shape': [4, 4]}
Range of reads: [1, 10000]


In [21]:
sampler=DWaveSampler(solver='DW_2000Q_6')
print("Couplings:",sampler.properties["couplers"])

Couplings: [[0, 4], [1, 4], [2, 4], [3, 4], [0, 5], [1, 5], [2, 5], [3, 5], [0, 6], [1, 6], [2, 6], [3, 6], [0, 7], [1, 7], [2, 7], [3, 7], [4, 12], [8, 12], [9, 12], [10, 12], [11, 12], [5, 13], [8, 13], [9, 13], [10, 13], [11, 13], [6, 14], [8, 14], [9, 14], [10, 14], [11, 14], [7, 15], [8, 15], [9, 15], [10, 15], [11, 15], [12, 20], [16, 20], [17, 20], [18, 20], [19, 20], [13, 21], [16, 21], [17, 21], [18, 21], [19, 21], [16, 22], [17, 22], [18, 22], [19, 22], [15, 23], [16, 23], [17, 23], [18, 23], [19, 23], [20, 28], [24, 28], [25, 28], [26, 28], [27, 28], [21, 29], [24, 29], [25, 29], [26, 29], [27, 29], [22, 30], [24, 30], [25, 30], [26, 30], [27, 30], [23, 31], [24, 31], [25, 31], [26, 31], [27, 31], [28, 36], [32, 36], [33, 36], [34, 36], [35, 36], [29, 37], [32, 37], [33, 37], [34, 37], [35, 37], [30, 38], [32, 38], [33, 38], [34, 38], [35, 38], [31, 39], [32, 39], [33, 39], [34, 39], [35, 39], [36, 44], [40, 44], [41, 44], [42, 44], [37, 45], [40, 45], [41, 45], [42, 45], [3

In [22]:
sampler.adjacency

{0: {4, 5, 6, 7, 128},
 1: {4, 5, 6, 7, 129},
 2: {4, 5, 6, 7, 130},
 3: {4, 5, 6, 7, 131},
 4: {0, 1, 2, 3, 12},
 5: {0, 1, 2, 3, 13},
 6: {0, 1, 2, 3, 14},
 7: {0, 1, 2, 3, 15},
 8: {12, 13, 14, 15, 136},
 9: {12, 13, 14, 15, 137},
 10: {12, 13, 14, 15, 138},
 11: {12, 13, 14, 15, 139},
 12: {4, 8, 9, 10, 11, 20},
 13: {5, 8, 9, 10, 11, 21},
 14: {6, 8, 9, 10, 11},
 15: {7, 8, 9, 10, 11, 23},
 16: {20, 21, 22, 23, 144},
 17: {20, 21, 22, 23, 145},
 18: {20, 21, 22, 23, 146},
 19: {20, 21, 22, 23, 147},
 20: {12, 16, 17, 18, 19, 28},
 21: {13, 16, 17, 18, 19, 29},
 22: {16, 17, 18, 19, 30},
 23: {15, 16, 17, 18, 19, 31},
 24: {28, 29, 30, 31, 152},
 25: {28, 29, 30, 31, 153},
 26: {28, 29, 30, 31, 154},
 27: {28, 29, 30, 31, 155},
 28: {20, 24, 25, 26, 27, 36},
 29: {21, 24, 25, 26, 27, 37},
 30: {22, 24, 25, 26, 27, 38},
 31: {23, 24, 25, 26, 27, 39},
 32: {36, 37, 38, 39, 160},
 33: {36, 37, 38, 39, 161},
 34: {36, 37, 38, 39, 162},
 35: {36, 37, 38, 39, 163},
 36: {28, 32, 33, 34, 

In [23]:
sampler=DWaveSampler(solver='Advantage_system4.1')
sampler.adjacency

{30: {31, 45, 2940, 2955, 2970, 2985},
 31: {30, 32, 46, 3120, 3135, 3150, 3165},
 32: {31, 33, 47, 3300, 3315, 3330, 3345},
 33: {32, 34, 48, 3480, 3495, 3510, 3525},
 34: {33, 35, 49, 3660, 3675, 3690, 3705},
 35: {34, 36, 50, 3840, 3855, 3870, 3885},
 36: {35, 37, 51, 4020, 4035, 4050, 4065},
 37: {36, 38, 52, 4200, 4215, 4230, 4245},
 38: {37, 39, 53, 4380, 4395, 4410, 4425},
 39: {38, 40, 54, 4560, 4575, 4590, 4605},
 40: {39, 41, 55, 4740, 4755, 4770, 4785},
 41: {40, 42, 56, 4920, 4935, 4950, 4965},
 42: {41, 43, 57, 5100, 5115, 5130, 5145},
 43: {42, 44, 58, 5280, 5295, 5310, 5325},
 44: {43, 59, 5460, 5475, 5490, 5505},
 45: {30, 46, 2940, 2955, 2970, 2985},
 46: {31, 45, 47, 3120, 3135, 3150, 3165},
 47: {32, 46, 48, 3300, 3315, 3330, 3345},
 48: {33, 47, 49, 3480, 3495, 3510, 3525},
 49: {34, 48, 50, 3660, 3675, 3690, 3705},
 50: {35, 49, 51, 3840, 3855, 3870, 3885},
 51: {36, 50, 52, 4020, 4035, 4050, 4065},
 52: {37, 51, 53, 4200, 4215, 4230, 4245},
 53: {38, 52, 54, 4380,

In [24]:
print(set([len(sampler.adjacency[i]) for i in sampler.adjacency]))

{6, 7, 10, 11, 12, 13, 14, 15}


### Embeddings

In [25]:
J = {(0,1):1, (0,2):1, (1,2):1}
h = {}
triangle = dimod.BinaryQuadraticModel(h, J, 0.0, dimod.SPIN)
# Embed it and solve it on the DW_2000Q_6 annealer
sampler = EmbeddingComposite(DWaveSampler(solver = "DW_2000Q_6"))
result = sampler.sample(triangle, num_reads=10, 
    return_embedding = True)
print("The samples obtained are")
print(result)
print("The embedding used was")
print(result.info["embedding_context"])

The samples obtained are
   0  1  2 energy num_oc. chain_.
0 -1 +1 -1   -1.0       1     0.0
1 +1 -1 +1   -1.0       1     0.0
2 +1 +1 -1   -1.0       1     0.0
3 -1 +1 +1   -1.0       5     0.0
4 -1 -1 +1   -1.0       2     0.0
['SPIN', 5 rows, 10 samples, 3 variables]
The embedding used was
{'embedding': {1: (981,), 0: (977,), 2: (979, 982)}, 'chain_break_method': 'majority_vote', 'embedding_parameters': {}, 'chain_strength': 1.9996979771955565}


##1.4- Annealing parameters

### Annealing time

In [26]:
sampler = DWaveSampler(solver = "Advantage_system4.1")
print("The default annealing time is",
    sampler.properties["default_annealing_time"],"microseconds")
print("The possible values for the annealing time (in microseconds)"\
    " lie in the range",sampler.properties["annealing_time_range"])

The default annealing time is 20.0 microseconds
The possible values for the annealing time (in microseconds) lie in the range [0.5, 2000.0]


In [27]:
J = {(0,1):1, (0,2):1, (1,2):1}
h = {}
triangle = dimod.BinaryQuadraticModel(h, J, 0.0, dimod.SPIN)
sampler = EmbeddingComposite(DWaveSampler(solver = "DW_2000Q_6"))
result = sampler.sample(triangle, num_reads=10, annealing_time = 100)
print("The samples obtained are")
print(result)

The samples obtained are
   0  1  2 energy num_oc. chain_.
0 -1 -1 +1   -1.0       2     0.0
1 -1 +1 -1   -1.0       1     0.0
2 +1 +1 -1   -1.0       1     0.0
3 +1 -1 +1   -1.0       2     0.0
4 +1 -1 -1   -1.0       1     0.0
5 -1 +1 +1   -1.0       3     0.0
['SPIN', 6 rows, 10 samples, 3 variables]


### Annealing schedules

In [28]:
forward_schedule=[[0.0, 0.0], [5.0, 0.25], [25, 0.75], [30, 1.0]]
sampler = EmbeddingComposite(DWaveSampler())
result = sampler.sample(triangle, num_reads=10, 
    anneal_schedule = forward_schedule)
print(result)

   0  1  2 energy num_oc. chain_.
0 -1 -1 +1   -1.0       1     0.0
1 -1 +1 -1   -1.0       2     0.0
2 -1 +1 +1   -1.0       1     0.0
3 +1 -1 -1   -1.0       1     0.0
4 +1 +1 -1   -1.0       1     0.0
5 +1 -1 +1   -1.0       4     0.0
['SPIN', 6 rows, 10 samples, 3 variables]


In [29]:
reverse_schedule=[[0.0, 1.0], [10.0, 0.5], [20, 1.0]]
initial_state = {0:1, 1:1, 2:1}
sampler = EmbeddingComposite(DWaveSampler())
result = sampler.sample(triangle, num_reads=10, 
    anneal_schedule = reverse_schedule,
    reinitialize_state=False, initial_state = initial_state)
print("The samples obtained are")
print(result)

The samples obtained are
   0  1  2 energy num_oc. chain_.
0 +1 -1 +1   -1.0       1     0.0
1 +1 -1 +1   -1.0       1     0.0
2 +1 +1 -1   -1.0       1     0.0
3 +1 -1 -1   -1.0       1     0.0
4 +1 -1 -1   -1.0       1     0.0
5 -1 -1 +1   -1.0       1     0.0
6 +1 -1 +1   -1.0       1     0.0
7 +1 +1 -1   -1.0       1     0.0
8 -1 -1 +1   -1.0       1     0.0
9 +1 -1 +1   -1.0       1     0.0
['SPIN', 10 rows, 10 samples, 3 variables]


### Coupling strength

In [30]:
sampler = DWaveSampler("Advantage_system4.1")
print("The coupling strength range is", sampler.properties["h_range"])

The coupling strength range is [-4.0, 4.0]


In [31]:
sampler = EmbeddingComposite(DWaveSampler("Advantage_system4.1"))
# Define the problem
x0 = dimod.Binary("x0")
x1 = dimod.Binary("x1")
x2 = dimod.Binary("x2")
blp = dimod.ConstrainedQuadraticModel()
blp.set_objective(-5*x0+3*x1-2*x2)
blp.add_constraint(x0 + x2 <= 1, "First constraint")
blp.add_constraint(3*x0 -x1 + 3*x2 <= 4, "Second constraint")
# Convert the problem and run it
qubo, invert = dimod.cqm_to_bqm(blp, lagrange_multiplier = 10)
result = sampler.sample(qubo, num_reads=100)
# Aggregate and show the results
samples = []
occurrences = []
for s in result.data():
    samples.append(invert(s.sample))
    occurrences.append(s.num_occurrences)
sampleset = dimod.SampleSet.from_samples_cqm(samples,blp,
    num_occurrences=occurrences)
print("The solutions to the original problem are")
print(sampleset.filter(lambda s: s.is_feasible).aggregate())

The solutions to the original problem are
  x0 x1 x2 energy num_oc. is_sat. is_fea.
0  1  0  0   -5.0      26 arra...    True
1  0  0  1   -2.0      24 arra...    True
2  1  1  0   -2.0      15 arra...    True
3  0  0  0    0.0      16 arra...    True
4  0  1  1    1.0      14 arra...    True
5  0  1  0    3.0       3 arra...    True
['INTEGER', 6 rows, 98 samples, 3 variables]


In [32]:
sampler = EmbeddingComposite(DWaveSampler("Advantage_system4.1"))
# Define the problem
x0 = dimod.Binary("x0")
x1 = dimod.Binary("x1")
x2 = dimod.Binary("x2")
blp = dimod.ConstrainedQuadraticModel()
blp.set_objective(-5*x0+3*x1-2*x2)
blp.add_constraint(x0 + x2 <= 1, "First constraint")
blp.add_constraint(3*x0 -x1 + 3*x2 <= 4, "Second constraint")
# Convert the problem and run it
qubo, invert = dimod.cqm_to_bqm(blp, lagrange_multiplier = 4)
result = sampler.sample(qubo, num_reads=100)
# Aggregate and show the results
samples = []
occurrences = []
for s in result.data():
    samples.append(invert(s.sample))
    occurrences.append(s.num_occurrences)
sampleset = dimod.SampleSet.from_samples_cqm(samples,blp,
    num_occurrences=occurrences)
print("The solutions to the original problem are")
print(sampleset.filter(lambda s: s.is_feasible).aggregate())

The solutions to the original problem are
  x0 x1 x2 energy num_oc. is_sat. is_fea.
0  1  0  0   -5.0      29 arra...    True
1  0  0  1   -2.0      13 arra...    True
2  1  1  0   -2.0      28 arra...    True
5  0  0  0    0.0       2 arra...    True
3  0  1  1    1.0      16 arra...    True
4  0  1  0    3.0       6 arra...    True
['INTEGER', 6 rows, 94 samples, 3 variables]


In [33]:
sampler = EmbeddingComposite(DWaveSampler("Advantage_system4.1"))
# Define the problem
x0 = dimod.Binary("x0")
x1 = dimod.Binary("x1")
x2 = dimod.Binary("x2")
blp = dimod.ConstrainedQuadraticModel()
blp.set_objective(-5*x0+3*x1-2*x2)
blp.add_constraint(x0 + x2 <= 1, "First constraint")
blp.add_constraint(3*x0 -x1 + 3*x2 <= 4, "Second constraint")
# Convert the problem and run it
qubo, invert = dimod.cqm_to_bqm(blp, lagrange_multiplier = 1)
result = sampler.sample(qubo, num_reads=100)
# Aggregate and show the results
samples = []
occurrences = []
for s in result.data():
    samples.append(invert(s.sample))
    occurrences.append(s.num_occurrences)
sampleset = dimod.SampleSet.from_samples_cqm(samples,blp,
    num_occurrences=occurrences)
print("The solutions to the original problem are")
print(sampleset.filter(lambda s: s.is_feasible).aggregate())

The solutions to the original problem are
  x0 x1 x2 energy num_oc. is_sat. is_fea.
0  1  0  0   -5.0      62 arra...    True
1  1  1  0   -2.0       3 arra...    True
2  0  0  1   -2.0      21 arra...    True
3  0  0  0    0.0       4 arra...    True
4  0  1  1    1.0       3 arra...    True
['INTEGER', 5 rows, 93 samples, 3 variables]


##1.5- Classic and hybrid solvers

In [34]:
import greedy
import dimod
# Define the problem
J = {(0,1):1, (1,2):1, (2,3):1, (3,0):1}
h = {}
problem = dimod.BinaryQuadraticModel(h, J, 0.0, dimod.SPIN)
# Sample with SteepestDescentSolver
solver = greedy.SteepestDescentSolver()
solution = solver.sample(problem, num_reads = 10)
print(solution.aggregate())

   0  1  2  3 energy num_oc. num_st.
0 -1 +1 -1 +1   -4.0       6       1
2 +1 -1 +1 -1   -4.0       3       1
1 -1 +1 +1 -1    0.0       1       0
['SPIN', 3 rows, 10 samples, 4 variables]


In [35]:
import tabu 
solver = tabu.TabuSampler()
solution = solver.sample(problem, num_reads = 10)
print(solution.aggregate())

   0  1  2  3 energy num_oc. num_re.
0 -1 +1 -1 +1   -4.0       3       1
1 +1 -1 +1 -1   -4.0       7       2
['SPIN', 2 rows, 10 samples, 4 variables]


In [36]:
import neal 
solver = neal.SimulatedAnnealingSampler()
solution = solver.sample(problem, num_reads = 10)
print(solution.aggregate())

   0  1  2  3 energy num_oc.
3 -1 +1 -1 +1   -4.0       1
6 +1 -1 +1 -1   -4.0       2
0 -1 +1 +1 +1    0.0       3
1 +1 +1 +1 -1    0.0       1
2 -1 -1 -1 +1    0.0       1
4 -1 -1 +1 +1    0.0       1
5 +1 -1 -1 -1    0.0       1
['SPIN', 7 rows, 10 samples, 4 variables]


In [37]:
import dwave.system 
sampler = dwave.system.LeapHybridSampler()
solution = solver.sample(problem, num_reads = 10)
print(solution.aggregate())

   0  1  2  3 energy num_oc.
0 +1 -1 +1 -1   -4.0       3
1 -1 -1 -1 +1    0.0       1
2 +1 +1 -1 -1    0.0       2
3 +1 -1 -1 -1    0.0       2
4 -1 +1 +1 +1    0.0       1
5 -1 -1 +1 +1    0.0       1
['SPIN', 6 rows, 10 samples, 4 variables]


In [38]:
sampler.properties["quota_conversion_rate"]

20